#### Import block

In [198]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import altair as alt
import geopy.distance as gpd
from scipy.sparse import coo_matrix

#### Function definitions

In [321]:
def one_hot_stack(px_combined,
                 centroid_lat = 0,
                 centroid_lon = 0,
                 centroid_date = dt.datetime.strptime('2017/01/01','%Y/%m/%d'),
                 span = 100,
                 days_back = 10,
                 days_forward = 5):
    
    min_date = centroid_date - dt.timedelta(days = days_back)
    max_date = centroid_date + dt.timedelta(days = days_forward)

    # Calculate location if we move 375 meters North and use to get grid steps.
    lat_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=0).latitude
    lat_step = abs(centroid_x - lat_375)

    # Calculate location if we move 375 meters East and use to get grid steps.
    lon_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=90).longitude
    lon_step = abs(centroid_y - lon_375)

    # Get bounding box of centroid and 100 pixels in all directions.
    min_lat = centroid_x - (lat_step * span)
    max_lat = centroid_x + (lat_step * span)
    min_lon = centroid_y - (lon_step * span)
    max_lon = centroid_y + (lon_step * span)

    # Filter data down to the range we want to plot
    t_df = px_combined[(px_combined['Date'] >= min_date) & (px_combined['Date'] <= max_date)]
    t_df = t_df[(t_df['Lat'] >= min_lat) & (t_df['Lat'] <= max_lat) & (t_df['Lon'] >= min_lon) & (t_df['Lon'] <= max_lon)]

    # Calculate pixel locations in the new grid.
    t_df['delta_lat'] = t_df['Lat'] - centroid_lat
    t_df['delta_lon'] = t_df['Lon'] - centroid_lon
    t_df['lat_grid'] = t_df['delta_lat'] // lat_step
    t_df['lon_grid'] = t_df['delta_lon'] // lon_step

    # Express date difference
    t_df['delta_day'] = (t_df['Date'] - centroid_date).dt.days
    
    # Convert grids to not have negative locations.
    t_df['lat_grid'] = t_df['lat_grid'] + span
    t_df['lon_grid'] = t_df['lon_grid'] + span
    
    # Build an array for each day in our date range then stack them. One for the inputs and one for the outputs.
    input_stack = []
    for x in range(-days_back, 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm = np.pad(cm, pad_width=((0, (2 * span + 1) - cm.shape[0]),(0,(2 * span + 1) - cm.shape[1])), mode = 'constant')
        else:
            cm = np.zeros(((2 * span + 1),(2 * span + 1)))
        input_stack.append(cm)
    input_stack = np.stack(input_stack)

    output_stack = []
    for x in range(1, days_forward + 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm = np.pad(cm, pad_width=((0, (2 * span + 1) - cm.shape[0]),(0,(2 * span + 1) - cm.shape[1])), mode = 'constant')
        else:
            cm = np.zeros(((2 * span + 1),(2 * span + 1)))
        output_stack.append(cm)
    output_stack = np.stack(output_stack)
    
    return input_stack, output_stack

In [318]:
def feature_stack(px_combined,
                 centroid_lat = 0,
                 centroid_lon = 0,
                 centroid_date = dt.datetime.strptime('2017/01/01','%Y/%m/%d'),
                 span = 100,
                 days_back = 10,
                 days_forward = 5):
    
    min_date = centroid_date - dt.timedelta(days = days_back)
    max_date = centroid_date + dt.timedelta(days = days_forward)

    # Calculate location if we move 375 meters North and use to get grid steps.
    lat_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=0).latitude
    lat_step = abs(centroid_x - lat_375)

    # Calculate location if we move 375 meters East and use to get grid steps.
    lon_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=90).longitude
    lon_step = abs(centroid_y - lon_375)

    # Get bounding box of centroid and 100 pixels in all directions.
    min_lat = centroid_x - (lat_step * span)
    max_lat = centroid_x + (lat_step * span)
    min_lon = centroid_y - (lon_step * span)
    max_lon = centroid_y + (lon_step * span)

    # Filter data down to the range we want to plot
    t_df = px_combined[(px_combined['Date'] >= min_date) & (px_combined['Date'] <= max_date)]
    t_df = t_df[(t_df['Lat'] >= min_lat) & (t_df['Lat'] <= max_lat) & (t_df['Lon'] >= min_lon) & (t_df['Lon'] <= max_lon)]

    # Calculate pixel locations in the new grid.
    t_df['delta_lat'] = t_df['Lat'] - centroid_lat
    t_df['delta_lon'] = t_df['Lon'] - centroid_lon
    t_df['lat_grid'] = t_df['delta_lat'] // lat_step
    t_df['lon_grid'] = t_df['delta_lon'] // lon_step

    # Express date difference
    t_df['delta_day'] = (t_df['Date'] - centroid_date).dt.days
    
    # Convert grids to not have negative locations.
    t_df['lat_grid'] = t_df['lat_grid'] + span
    t_df['lon_grid'] = t_df['lon_grid'] + span
    
    # Build an array for each day in our date range then stack them. One for the inputs and one for the outputs.
    input_stack = []
    for x in range(-days_back, 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm1 = coo_matrix((s_df['T4'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm1 = np.pad(cm1, pad_width=((0, 201 - cm1.shape[0]),(0,(2 * span + 1) - cm1.shape[1])), mode = 'constant')
            cm2 = coo_matrix((s_df['T5'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm2 = np.pad(cm2, pad_width=((0, 201 - cm2.shape[0]),(0,(2 * span + 1) - cm2.shape[1])), mode = 'constant')
            cm3 = coo_matrix((s_df['FRP'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm3 = np.pad(cm3, pad_width=((0, 201 - cm3.shape[0]),(0,(2 * span + 1) - cm3.shape[1])), mode = 'constant')
            cm4 = coo_matrix((s_df['Rad13'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm4 = np.pad(cm4, pad_width=((0, 201 - cm4.shape[0]),(0,(2 * span + 1) - cm4.shape[1])), mode = 'constant')
        else:
            cm1 = np.zeros(((2 * span + 1),(2 * span + 1)))
            cm2 = np.zeros(((2 * span + 1),(2 * span + 1)))
            cm3 = np.zeros(((2 * span + 1),(2 * span + 1)))
            cm4 = np.zeros(((2 * span + 1),(2 * span + 1)))
        input_stack.append(np.stack([cm1, cm2, cm3, cm4]))
    input_stack = np.stack(input_stack)

    output_stack = []
    for x in range(1, days_forward + 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm = np.pad(cm, pad_width=((0, (2 * span + 1) - cm.shape[0]),(0,(2 * span + 1) - cm.shape[1])), mode = 'constant')
        else:
            cm = np.zeros(((2 * span + 1),(2 * span + 1)))
        output_stack.append(cm)
    output_stack = np.stack(output_stack)
    
    return input_stack, output_stack

#### Data import

In [44]:
# Load saved data back into memory.
px_combined = pd.read_csv('pixel_data_filtered.csv')

In [47]:
# Adjust date format.
px_combined['Date'] = pd.to_datetime(px_combined['YYYYMMDD'].astype(str))

In [ ]:
# Add a column where every value is a 1 to use in 1-hot encoding strategy.
px_combined['value'] = 1

In [123]:
# Limit to night pixels only.
px_combined = px_combined[px_combined['DNFlag'] == 'night']

#### Testing code concepts on one example

In [217]:
centroid_lat = -17.859758
centroid_lon = -57.332260
centroid_date = dt.datetime.strptime('2020/07/06','%Y/%m/%d')
days_back = 10
days_forward = 5
min_date = centroid_date - dt.timedelta(days = days_back)
max_date = centroid_date + dt.timedelta(days = days_forward)

# Calculate location if we move 375 meters North and use to get grid steps.
lat_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=0).latitude
lat_step = abs(centroid_x - lat_375)

# Calculate location if we move 375 meters East and use to get grid steps.
lon_375 = gpd.distance(kilometers=0.375).destination((centroid_x, centroid_y), bearing=90).longitude
lon_step = abs(centroid_y - lon_375)

# Get bounding box of centroid and 100 pixels in all directions.
min_lat = centroid_x - (lat_step * 100)
max_lat = centroid_x + (lat_step * 100)
min_lon = centroid_y - (lon_step * 100)
max_lon = centroid_y + (lon_step * 100)

# Filter data down to the range we want to plot
t_df = px_combined[(px_combined['Date'] >= min_date) & (px_combined['Date'] <= max_date)]
t_df = t_df[(t_df['Lat'] >= min_lat) & (t_df['Lat'] <= max_lat) & (t_df['Lon'] >= min_lon) & (t_df['Lon'] <= max_lon)]

# Calculate pixel locations in the new grid.
t_df['delta_lat'] = t_df['Lat'] - centroid_lat
t_df['delta_lon'] = t_df['Lon'] - centroid_lon
t_df['lat_grid'] = t_df['delta_lat'] // lat_step
t_df['lon_grid'] = t_df['delta_lon'] // lon_step

# Express date difference
t_df['delta_day'] = (t_df['Date'] - centroid_date).dt.days

In [218]:
# Add tooltip for plot
t_df['tool'] = t_df['lon_grid'].astype(str) + ' x ' + t_df['lat_grid'].astype(str)

# Show new grid in a plot.
alt.Chart(t_df[['lat_grid', 'lon_grid', 'Date', 'tool']]).mark_circle(size=250).encode(
    alt.X('lon_grid', title = 'Lon', axis=alt.Axis(labels=False)),
    alt.Y('lat_grid', title = 'Lat', axis=alt.Axis(labels=False)),
    color = alt.Color('Date:N', scale=alt.Scale(scheme = 'category20')),
    tooltip = 'tool'
).properties(height=600, width=600).interactive()

alt.Chart(...)

In [219]:
# Illustrate what we are asking model to predict.
source = alt.Chart(t_df[t_df['Date'] <= centroid_date][['lat_grid', 'lon_grid', 'Date', 'tool']]
                  ,title = 'Source Data').mark_circle(size=250).encode(
    alt.X('lon_grid', title = 'Lon', axis=alt.Axis(labels=False), scale=alt.Scale(domain=[-100, 100])),
    alt.Y('lat_grid', title = 'Lat', axis=alt.Axis(labels=False), scale=alt.Scale(domain=[-100, 100])),
    color = alt.Color('Date:N', scale=alt.Scale(scheme = 'category20')),
    tooltip = 'tool'
).properties(height=300, width=300).interactive()

target = alt.Chart(t_df[t_df['Date'] > centroid_date][['lat_grid', 'lon_grid', 'Date', 'tool']]
                  ,title = 'Target Prediction').mark_circle(size=250).encode(
    alt.X('lon_grid', title = 'Lon', axis=alt.Axis(labels=False), scale=alt.Scale(domain=[-100, 100])),
    alt.Y('lat_grid', title = 'Lat', axis=alt.Axis(labels=False), scale=alt.Scale(domain=[-100, 100])),
    color = alt.Color('Date:N', scale=alt.Scale(scheme = 'category20')),
    tooltip = 'tool'
).properties(height=300, width=300).interactive()

source | target

alt.HConcatChart(...)

### One-hot encoding strategy

In [223]:
# Convert grids to not have negative locations.
t_df['lat_grid'] = t_df['lat_grid'] + 100
t_df['lon_grid'] = t_df['lon_grid'] + 100

In [251]:
# Examine array for centroid date.
s_df = t_df[t_df['Date'] == centroid_date]
cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
cm = np.pad(cm, pad_width=((0, 201 - cm.shape[0]),(0,201 - cm.shape[1])), mode = 'constant')

# Confirm that the right number of pixels are mapped.
pd.DataFrame(cm).sum().sum()

6

In [252]:
s_df

,Unnamed: 0.1,Unnamed: 0,YYYYMMDD,HHMM,sat,Line,Sample,Lat,Lon,T4,...,Type,PixArea,Date,value,delta_lat,delta_lon,lat_grid,lon_grid,delta_day,tool
6219262,16125,319850,20200706,412,VNP,5210,530,-17.852758,-57.335457,298.63,...,0,0.321,2020-07-06,1,0.007000,-0.003197,102.0,99.0,0,-1.0 x 2.0
6219263,16126,319851,20200706,412,VNP,5211,531,-17.859758,-57.332260,299.43,...,0,0.320,2020-07-06,1,0.000000,0.000000,100.0,100.0,0,0.0 x 0.0
6219445,16308,324131,20200706,554,VNP,3511,6256,-17.867916,-57.319241,325.20,...,0,0.505,2020-07-06,1,-0.008158,0.013019,97.0,103.0,0,3.0 x -3.0
6219446,16309,324132,20200706,554,VNP,3511,6255,-17.867306,-57.325962,313.15,...,0,0.504,2020-07-06,1,-0.007548,0.006298,97.0,101.0,0,1.0 x -3.0
6219450,16313,324136,20200706,554,VNP,3528,6256,-17.866890,-57.324802,321.05,...,0,0.505,2020-07-06,1,-0.007132,0.007458,97.0,102.0,0,2.0 x -3.0
6219451,16314,324137,20200706,554,VNP,3528,6257,-17.867458,-57.318073,316.99,...,0,0.506,2020-07-06,1,-0.007700,0.014187,97.0,104.0,0,4.0 x -3.0


In [264]:
# Build an array for each day in our date range then stack them. One for the inputs and one for the outputs.
input_stack = []
for x in range(-days_back, 1):
    s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
    if len(s_df) > 0:
        cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
        cm = np.pad(cm, pad_width=((0, 201 - cm.shape[0]),(0,201 - cm.shape[1])), mode = 'constant')
    else:
        cm = np.zeros((201,201))
    input_stack.append(cm)
input_stack = np.stack(input_stack)

output_stack = []
for x in range(1, days_forward + 1):
    s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
    if len(s_df) > 0:
        cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
        cm = np.pad(cm, pad_width=((0, 201 - cm.shape[0]),(0,201 - cm.shape[1])), mode = 'constant')
    else:
        cm = np.zeros((201,201))
    output_stack.append(cm)
output_stack = np.stack(output_stack)

In [273]:
print('Input Stack Shape:', input_stack.shape)
print('Input Stack Sum:', input_stack.sum())
print('Outpt Stack Shape:', output_stack.shape)
print('Output Stack Sum:', output_stack.sum())
print('Validation Check:', input_stack.sum() + output_stack.sum() == t_df.value.sum())

Input Stack Shape: (11, 201, 201)
Input Stack Sum: 72.0
Outpt Stack Shape: (5, 201, 201)
Output Stack Sum: 73.0
Validation Check: True


In [322]:
# Test function call.
i,o = one_hot_stack(px_combined, 
                    centroid_lat = -17.859758, 
                    centroid_lon = -57.332260, 
                    centroid_date = dt.datetime.strptime('2020/07/06','%Y/%m/%d'))
print('Input Stack Shape:', i.shape)
print('Input Stack Sum:', i.sum())
print('Outpt Stack Shape:', o.shape)
print('Output Stack Sum:', o.sum())

Input Stack Shape: (11, 201, 201)
Input Stack Sum: 72.0
Outpt Stack Shape: (5, 201, 201)
Output Stack Sum: 73.0


### Feature-based encoding strategy

In [315]:
# Create arrays for each feature
s_df = t_df[t_df['Date'] == centroid_date]
cm1 = coo_matrix((s_df['T4'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
cm1 = np.pad(cm1, pad_width=((0, 201 - cm1.shape[0]),(0,201 - cm1.shape[1])), mode = 'constant')
cm2 = coo_matrix((s_df['T5'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
cm2 = np.pad(cm2, pad_width=((0, 201 - cm2.shape[0]),(0,201 - cm2.shape[1])), mode = 'constant')
cm3 = coo_matrix((s_df['FRP'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
cm3 = np.pad(cm3, pad_width=((0, 201 - cm3.shape[0]),(0,201 - cm3.shape[1])), mode = 'constant')
cm4 = coo_matrix((s_df['Rad13'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
cm4 = np.pad(cm4, pad_width=((0, 201 - cm4.shape[0]),(0,201 - cm4.shape[1])), mode = 'constant')

np.stack([cm1,cm2,cm3,cm4]).shape

(4, 201, 201)

In [319]:
# Test function call.
i,o = feature_stack(px_combined, 
                    centroid_lat = -17.859758, 
                    centroid_lon = -57.332260, 
                    centroid_date = dt.datetime.strptime('2020/07/06','%Y/%m/%d'))
print('Input Stack Shape:', i.shape)
print('Input Stack Sum:', i.sum())
print('Outpt Stack Shape:', o.shape)
print('Output Stack Sum:', o.sum())

Input Stack Shape: (11, 4, 201, 201)
Input Stack Sum: 43676.83200000001
Outpt Stack Shape: (5, 201, 201)
Output Stack Sum: 73.0
